In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import shutil
import re
import time
import json
import unicodedata
import html
import re
from functools import reduce

In [62]:
def clean_text(text):
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")
    text = text.replace(u'\xa0', u' ')
    text.strip()
    return text



In [66]:
main_url = "http://www.gardening.cornell.edu/homegardening/sceneb771.html"
base_url = "http://www.gardening.cornell.edu/homegardening/"
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')
html_pages = []
for link in soup.find_all('a'):
    html_pages.append(link.get('href'))
formatted_pages = []
for page in html_pages:
    if page != None and "scene" in str(page):
        formatted = base_url + page
        formatted_pages.append(formatted)

In [81]:
final_dict = {}
for url in formatted_pages:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    bolds = soup.find_all(['b'])
    name = bolds[1].text.strip()
    
    data_dict = {}
    headers = soup.find_all(['p', 'li', 'b'])
    total_text = []
    for head in headers:
        total_text.append(head.text)

    total_text = list(map(lambda t : clean_text(t), total_text))
    occurs1 = occurs2 = occurs3 = occurs4 = occurs5 = occurs6 = occurs7 = occurs8 = True
    for text in total_text:
        if "Germination temperature:" in text and occurs1:
            text = text.replace("Germination temperature:", "").strip()
            data_dict["Germ_Temp"] = text
            occurs1 = False
        if "Days to emergence:" in text and occurs2:
            data_dict["Germ_Time"] = text.strip()
            occurs2 = False
        if "Height:" in text and occurs3:
            text = text.replace("Height:", "").strip()
            data_dict["Height"] = text
            occurs3 = False
        if "Spread:" in text and occurs4:
            text = text.replace("Spread:", "").strip()
            data_dict["Spread"] = text
            occurs4 = False
        if "Lifecycle:" in text and occurs5:
            text = text.replace("Lifecycle:", "").strip()
            data_dict["Lifecycle"] = text
            occurs5 = False
        if "Ease-of-care:" in text and occurs6:
            text = text.replace("Ease-of-care:", "").strip()
            data_dict["Care"] = text
            occurs6 = False
        if "sun" in text and occurs7:
            data_dict["Sun"] = text
            occurs7 = False
        if "Soil Conditions" in text and occurs8:
            text = text.replace("Soil Conditions:", "").strip()
            data_dict["Soil"] = text
            occurs8 = False


    combined_text = reduce(lambda s, acc : acc + s, total_text)
    combined_text = combined_text.replace("\n", ";;")
    data_dict["Combined"] = combined_text
    
    final_dict[name] = data_dict
    

In [80]:
fp = open("initial_plant_scrape.json", "w")
json.dump(final_dict, fp)
fp.close()